Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cobra
import escher

# Load model

Choose from alternatives

In [ ]:
# Yeast 8
model = cobra.io.read_sbml_model("../data/gemfiles/yeast-GEM-BiGG.xml")

In [ ]:
# Enzyme-constrained Yeast 8, batch
# https://github.com/SysBioChalmers/ecModels/tree/main/ecYeastGEM/model
# This is supposed under CI, i.e.
# automatically re-generated and updated when new models are available.
# This model is based on Yeast8.3.4

# Average enzyme saturation factor (sigma) = 0.5
# Total protein content in the cell [g protein/gDw] (Ptot) = 0.5
# Fraction of enzymes in the model [g enzyme/g protein] (f) = 0.5
# https://github.com/SysBioChalmers/GECKO/blob/main/userData/ecYeastGEM/YeastGEMAdapter.m
model = cobra.io.read_sbml_model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")

Show model

In [ ]:
model

# Objective function

In the ecYeast8 (batch) model, the objective function -- growth -- is reaction ID `r_2111`.

This reaction is linked to the biomass reaction, ID `r_4041`.

Here, we also see the stoichiometry.  There are five classes of macromolecules: lipids, proteins, carbohydrates, DNA, and RNA.  And there are two other bulk metabolites: cofactor and ion.

In [ ]:
model.reactions.get_by_id('r_2111')

In [ ]:
model.reactions.get_by_id('r_4041')

Medium

In [ ]:
model.medium

In [ ]:
for reaction_id in model.medium.keys():
    print(model.reactions.get_by_id(reaction_id).name)

Remove bounds on glucose uptake and growth rate

In [ ]:
# (no need because bounds are already unrestricted)
# Unrestrict glucose uptake
model.reactions.get_by_id('r_1714').bounds = (-1000.0, 0)
# Unrestrict oxygen uptake (aerobic)
model.reactions.get_by_id('r_1992').bounds = (-1000.0, 0)
# Unrestrict objective function
model.reactions.get_by_id('r_4041').bounds = (0, 1000.0)

Optimise using (vanilla) FBA

In [ ]:
solution = model.optimize()

In [ ]:
model.summary()

In [ ]:
solution['r_0466No1']

Linear reaction coefficients

In [ ]:
cobra.util.solver.linear_reaction_coefficients(model)

# Modify biomass reaction by ablating each type of macromolecule

## Some convenience classes

In [ ]:
CELL_DRY_MASS = 15e-12 # g
MW_BIOMASS_MMOL = 979.24108756487 / 1000

class BiomassComponent():
    def __init__(
        self,
        metabolite_label,
        metabolite_id,
        pseudoreaction,
        molecular_mass,
        mass_per_cell,
        copy_number,
    ):
        self.metabolite_label = metabolite_label
        self.metabolite_id = metabolite_id
        self.pseudoreaction = pseudoreaction
        self.molecular_mass = molecular_mass # g/mmol
        self.mass_per_cell = mass_per_cell # g
        self.copy_number = copy_number
        
        self.ablated_flux = None # h-1
        self.est_time = None # h
        
    def get_est_time(self):
        self.est_time = (self.molecular_mass/MW_BIOMASS_MMOL) * (np.log(2)/self.ablated_flux)

We use molecular weights calculated earlier and other attributes from the cell economics project.

In [ ]:
MW_CARB = 368.03795704972003
MW_DNA = 3.9060196439999997
MW_RNA = 64.04235752722991
MW_PROTEIN = 504.3744234012359
MW_COFACTOR = 4.832782477018401
MW_ION = 2.4815607543700002
MW_LIPID = 31.5659867112958
MW_BIOMASS = 979.24108756487

In [ ]:
# TODO:
# - Create CSV table containing these
# - Create a class builder that builds these classes based on the CSV table
# - FURTHER: make it able to deal with ranges of values (lower limit, upper limit)

Lipids = BiomassComponent(
    metabolite_label='lipid',
    metabolite_id='s_1096[c]',
    pseudoreaction='r_2108',
    molecular_mass=MW_LIPID*1e-3,
    mass_per_cell=900e-15,
    copy_number=1e9,
)

Proteins = BiomassComponent(
    metabolite_label='protein',
    metabolite_id='s_3717[c]',
    pseudoreaction='r_4047',
    molecular_mass=MW_PROTEIN*1e-3,
    mass_per_cell=7650e-15,
    copy_number=1e8,
)

Carbohydrates = BiomassComponent(
    metabolite_label='carbohydrate',
    metabolite_id='s_3718[c]',
    pseudoreaction='r_4048',
    molecular_mass=MW_CARB*1e-3,
    mass_per_cell=(75+3450)*1e-15, # 'storage carbohydrates' + 'structural polymers'
    copy_number=2122804981, # estimated from above & avogadro's const
)

DNA = BiomassComponent(
    metabolite_label='DNA',
    metabolite_id='s_3720[c]',
    pseudoreaction='r_4050',
    molecular_mass=MW_DNA*1e-3,
    mass_per_cell=75e-15,
    copy_number=16,
)

RNA = BiomassComponent(
    metabolite_label='RNA',
    metabolite_id='s_3719[c]',
    pseudoreaction='r_4049',
    molecular_mass=MW_RNA*1e-3,
    mass_per_cell=1650e-15,
    copy_number=4e6,
)

Cofactors = BiomassComponent(
    metabolite_label='cofactor',
    metabolite_id='s_4205[c]',
    pseudoreaction='r_4598',
    molecular_mass=MW_COFACTOR*1e-3,
    mass_per_cell=1,
    copy_number=1,
)

Ions = BiomassComponent(
    metabolite_label='ion',
    metabolite_id='s_4206[c]',
    pseudoreaction='r_4599',
    molecular_mass=MW_ION*1e-3,
    mass_per_cell=1,
    copy_number=1,
)

Copy model

In [ ]:
genes_to_delete = ['ZWF1', 'ALD6', 'IDP2']
for gene in genes_to_delete:
    model.genes.get_by_id(gene_systematic[gene]).knock_out()

In [ ]:
model_saved = model.copy()

Simulate

In [ ]:
from cobra.util.solver import linear_reaction_coefficients

# Set up lists
biomass_component_list = [Lipids, Proteins, Carbohydrates, DNA, RNA, Cofactors, Ions]

all_metabolite_ids = [
    biomass_component.metabolite_id
    for biomass_component in biomass_component_list
]

all_pseudoreaction_ids = [
    (biomass_component.metabolite_label, biomass_component.pseudoreaction)
    for biomass_component in biomass_component_list
]
all_pseudoreaction_ids.append(('biomass', 'r_4041'))
all_pseudoreaction_ids.append(('objective', 'r_2111'))
    
def barplot_fluxes(pfba_solution, all_pseudoreactions_ids, plot_title):
    plt.subplots()
    plt.bar(
        [label for (label, _) in all_pseudoreaction_ids],
        [pfba_solution.fluxes[pseudoreaction_id]
         for (_, pseudoreaction_id) in all_pseudoreaction_ids],
    )
    plt.ylim((0,3))
    plt.title(plot_title)
    plt.xticks(rotation=45, ha="right")
    
# ORIGINAL

model = model_saved.copy()
# Unrestrict glucose uptake
model.reactions.get_by_id('r_1714').bounds = (-1000, 0)
# Unrestrict objective function
model.reactions.get_by_id('r_2111').bounds = (0, 1000)
# Optimise using FBA
fba_solution = model.optimize()

# Outputs
barplot_fluxes(fba_solution, all_pseudoreaction_ids, plot_title='original')
wt_growth_flux = fba_solution.fluxes["r_2111"]
print(f'Flux: {wt_growth_flux} h-1')
biomass_time = np.log(2)/(wt_growth_flux)
print(f'Estimated time: {biomass_time:.4f} hours')
print('\n')

list_component_times = []
for biomass_component in biomass_component_list:
    component_time = (biomass_component.molecular_mass/MW_BIOMASS_MMOL) * (np.log(2)/wt_growth_flux)
    list_component_times.append(component_time)
    print(f'Time for {biomass_component.metabolite_label}:\t\t {component_time:.4f} hours')
print(f'Sum: {sum(list_component_times):.4f}')
print('\n')

# ABLATED

for biomass_component in biomass_component_list:
    print(f'Prioritising {biomass_component.metabolite_label}')
    model = model_saved.copy()
    model.reactions.get_by_id('r_1714').bounds = (-1000, 0)
    model.reactions.get_by_id('r_2111').bounds = (0, 1000)
    
    # boilerplate: lookup
    to_ablate = all_metabolite_ids.copy()
    to_ablate.remove(biomass_component.metabolite_id)
    to_ablate_keys = [
        model.metabolites.get_by_id(metabolite_id)
        for metabolite_id in to_ablate
    ]
    to_ablate_dict = dict(zip(to_ablate_keys, [-1]*len(to_ablate_keys)))
    
    # ablate metabolites from biomass reaction
    model.reactions.get_by_id('r_4041').subtract_metabolites(to_ablate_dict)
    # re-scale stoichiometry of pseudometabolite of interest
    scaling_factor = MW_BIOMASS_MMOL/biomass_component.molecular_mass
    model.reactions.get_by_id('r_4041').add_metabolites({
        model.metabolites.get_by_id(biomass_component.metabolite_id): -(scaling_factor - 1)
    })
    print(model.reactions.get_by_id('r_4041').reaction)
    
    # optimise model
    fba_solution = model.optimize()
    
    # Outputs
    biomass_component.ablated_flux = fba_solution.fluxes["r_2111"]

    barplot_fluxes(
        fba_solution,
        all_pseudoreaction_ids,
        plot_title=f'Prioritising {biomass_component.metabolite_label}'
    )
    print(f'Flux: {fba_solution.fluxes["r_2111"]} h-1')
    biomass_component.get_est_time()
    print(f'Estimated time: {biomass_component.est_time:.4f} hours')
    print('\n')
    
total_time = sum([biomass_component.est_time for biomass_component in biomass_component_list])
print(f'sum of times: {total_time:.4f} hours')